#### LIVRABLE PROJET PYTHON

#### INPUT
1) Import des libraires

In [2]:
import os
from datetime import datetime 
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objects as go

2) Création des dataframes à partir des fichiers csv

In [3]:
def load_data_from_csv():

    notebook_dir = os.getcwd()                          #Chemin absolu du notebook
    data_dir = os.path.join(notebook_dir, "../data")    #Construction du chemin absolu vers le dossier 'data'
    csv_files = os.listdir(data_dir)                    #Liste des fichiers dans le dossier 'data'

    dataframes_dict = {}                                #Déclaration dictionnaire pour contenir les dataframes
    
    for file in csv_files:                              #Boucle pour chaque fichier .csv du dossier 'data' : 
        if file.endswith('.csv'):
            
            file_path = os.path.join(data_dir, file) 
            df = pd.read_csv(file_path)                         #Création du dataframe

            file_name = os.path.splitext(file)[0]
            dataframes_dict['df_' + str(file_name)] = df        #Renommage du dataframe avec préfixe "df_" + 'nom_du_fichier'
            
            df['timestamp'] = pd.to_datetime(df['timestamp'])   #Conversion 'timestamp' en type Datetime    

            print(f"df_{file_name}")                            #Affichage du df créé pour vérification
            print(df.head(3))   
            print("\n")

    return dataframes_dict

dataframes_dict = load_data_from_csv()

df_stmp
            timestamp    price  amount
0 2021-02-24 23:59:54  49754.0   0.753
1 2021-02-24 23:59:52  49754.0   0.116
2 2021-02-24 23:59:52  49754.0   0.104


df_lmax
                timestamp    price  amount
0 2021-02-24 23:59:59.691  49767.0    0.01
1 2021-02-24 23:59:42.786  49752.0    0.06
2 2021-02-24 23:59:42.785  49752.0    0.30


df_gmni
                timestamp     price    amount
0 2021-02-24 23:59:53.406  49773.07  0.003435
1 2021-02-24 23:59:53.406  49766.06  0.051690
2 2021-02-24 23:59:47.280  49746.16  0.122833


df_itbi
                timestamp     price  amount
0 2021-02-24 23:59:48.157  49753.50  0.0001
1 2021-02-24 23:59:45.463  49753.75  0.0004
2 2021-02-24 23:59:38.887  49734.50  0.0001


df_okcn
                timestamp     price  amount
0 2021-02-24 23:59:57.847  49724.93    0.02
1 2021-02-24 23:59:49.743  49730.33    0.02
2 2021-02-24 23:59:35.623  49706.87    0.02


df_bfnx
                timestamp    price  amount
0 2021-02-24 23:59:58.181  49716.0 

#### MANIPULATION DES DONNEES

1) Création d'un dataframe 'df_all' et intégration dans le dictionnaire

In [4]:

def global_df_creation(): 
    
    dfs_to_concat = list(dataframes_dict.values())      #Identification des dataframes à concaténer
    print(f"Concaténation des dataframes :\n{list(dataframes_dict)}\n")
    
    df_all = pd.concat(dfs_to_concat, ignore_index=True)        #Concaténation verticale -> pas de perte de données avec la fonction concat, redéfinition des index pour avoir une clé unique par transaction
    print("'df_all' :")     
    display(df_all)     #Vérification : '2021-02-24 23:59:52.000' *2 dans le display = transactions conservées      

    dataframes_dict['df_all']= df_all       #Intégration de 'df_all' dans le dictionnaire
    print(f"Intégration dans le dictionnaire :\n{list(dataframes_dict)}\n")     
    
global_df_creation() 


Concaténation des dataframes :
['df_stmp', 'df_lmax', 'df_gmni', 'df_itbi', 'df_okcn', 'df_bfnx', 'df_btrx', 'df_bfly', 'df_bnus', 'df_cbse', 'df_krkn']

'df_all' :


,timestamp,price,amount
0,2021-02-24 23:59:54.000,49754.0,0.753000
1,2021-02-24 23:59:52.000,49754.0,0.116000
2,2021-02-24 23:59:52.000,49754.0,0.104000
3,2021-02-24 23:59:49.000,49754.0,0.016000
4,2021-02-24 23:59:45.000,49754.0,0.011000
...,...,...,...
1136788,2021-02-24 00:00:11.182,48899.8,0.023270
1136789,2021-02-24 00:00:10.373,48899.9,0.200000
1136790,2021-02-24 00:00:07.818,48899.9,0.018278
1136791,2021-02-24 00:00:02.351,48899.9,0.002045


Intégration dans le dictionnaire :
['df_stmp', 'df_lmax', 'df_gmni', 'df_itbi', 'df_okcn', 'df_bfnx', 'df_btrx', 'df_bfly', 'df_bnus', 'df_cbse', 'df_krkn', 'df_all']



#### CALCUL

1) Agrégation des données et calcul selon paramètres sélectionnés

In [5]:
# Widgets globaux
def setup_widgets():
    dropdown_ladder = widgets.Dropdown(options=['60min', '30min', '5min'], description='Interval:', value='60min')
    dropdown_vwmp_type = widgets.Dropdown(options=['lower', 'upper'], description='VWMP:', value='lower')
    export_button = widgets.Button(description="Exporter en CSV")

    export_button.on_click(lambda b: export_csv(dropdown_ladder.value, dropdown_vwmp_type.value))

    return dropdown_ladder, dropdown_vwmp_type, export_button

# Agrégation et traitement des données
def process_data(dataframes_dict, frequency, vwmp_type):
    aggregated_dict = {}

    for key, df in dataframes_dict.items():
        exchange_name = key.split("df_")[1]
        aggregated_df = aggregate_data(df, frequency)
        cleaned_df = clean_data(aggregated_df)
        processed_df = calculate_metrics(cleaned_df, frequency, vwmp_type, exchange_name)
        aggregated_dict[key] = processed_df

    return aggregated_dict

def aggregate_data(df, frequency):
    df['weighted_volume'] = df['price'] * df['amount']
    aggregated_df = df.groupby(pd.Grouper(key='timestamp', freq=frequency)).agg({
        'price': ['sum', 'first', 'max', 'min', 'last'],
        'amount': 'sum',
        'weighted_volume': 'sum'
    })

    aggregated_df.columns = ['price', 'price_open', 'price_high', 'price_low', 'price_close', 'amount', 'weighted_volume']
    return aggregated_df

def clean_data(df):
    df_cleaned = df.dropna(how='all')
    return df_cleaned

def calculate_metrics(df, frequency, vwmp_type, exchange_name):
    df[f'{exchange_name}_vwap'] = df.groupby(pd.Grouper(key='timestamp', freq=frequency))['weighted_volume'].transform('sum') / df.groupby(pd.Grouper(key='timestamp', freq=frequency))['amount'].transform('sum')
    df['ecart_type'] = df.groupby(pd.Grouper(key='timestamp', freq=frequency))['price'].transform(np.nanstd)
    
    series_sorted = df.sort_values('amount')
    total_volume_median = series_sorted['amount'].cumsum().max() / 2
    
    if vwmp_type == 'lower':
        vwmp = series_sorted.loc[series_sorted['cumul_amount'] <= total_volume_median, 'price'].iloc[-1] if not series_sorted.empty else df['price'].mean()
    elif vwmp_type == 'upper':
        vwmp = series_sorted.loc[series_sorted['cumul_amount'] >= total_volume_median, 'price'].iloc[0] if not series_sorted.empty else df['price'].mean()
    else:
        raise ValueError("Mode non valide. Veuillez spécifier 'lower' ou 'upper'.")
    
    df[f'vwmp_{vwmp_type}'] = vwmp

    return df

# Visualisation
def visualize_data(aggregated_dict):
    for key, df in reversed(list(aggregated_dict.items())):
        exchange_name = key.split("df_")[1]

        candlestick = go.Figure(data=[
            go.Candlestick(x=df.index, open=df['price_open'], high=df['price_high'], low=df['price_low'], close=df['price_close'])
        ])
        
        candlestick.update_layout(title=f'Exchange - [{exchange_name}]', yaxis_title='Price', height=300, width=800, margin=dict(l=70, r=50, t=50, b=20))
        candlestick.add_trace(go.Scatter(x=df.index, y=df[f'{exchange_name}_vwap'], mode='lines', name='VWAP', line_color='blue'))
        candlestick.add_trace(go.Scatter(x=df.index, y=df[f'vwmp_lower'], mode='lines', name='VWMP Lower', line_color='grey'))
        candlestick.show()

# Export CSV
def export_csv(frequency, vwmp_type):
    export_folder = os.path.join(os.getcwd(), f"../output/Export_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}_freq-{frequency}_vwmp-{vwmp_type}")
    os.makedirs(export_folder)

    for key, df in processed_dict.items():
        filename = f"{key}.csv"
        file_path = os.path.join(export_folder, filename)
        df.to_csv(file_path, index=True)
    
    print(f"Export des fichiers CSV avec succès : {export_folder}")

# Application principale
def main(dataframes_dict):
    dropdown_ladder, dropdown_vwmp_type, export_button = setup_widgets()

    def update_data(change):
        clear_output()
        processed_dict = process_data(dataframes_dict, dropdown_ladder.value, dropdown_vwmp_type.value)
        visualize_data(processed_dict)

    dropdown_ladder.observe(update_data, names='value')
    dropdown_vwmp_type.observe(update_data, names='value')

    display(dropdown_ladder, dropdown_vwmp_type, export_button)
    update_data(None)  # Charger les données initiales au démarrage

# Exécution principale
if __name__ == "__main__":
    # Charger les données initiales (dataframes_dict)
    #dataframes_dict = load_data_from_csv()  # Assurez-vous d'avoir cette fonction définie

    # Démarrer l'application
    main(dataframes_dict)

KeyError: 'The grouper name timestamp is not found'

#### RESULTATS

Dans le cas où les widgets ne fonctionnerait pas dans le jupyter notebook :

1) Lancer l'aggrégation avec les paramètres souhaités et visualiser les résultats :

In [ ]:
aggregation('60min', 'lower')   

    #agg1 : ('60min', '30min', '5min')
    #agg2 : ('lower', 'upper')

2) Exporter les données avec les paramètres sélectionnés

In [ ]:
b =True
on_export_button_click(b)

Arborescence du projet : 

In [ ]:
##  .
##  ├── data
##  │   ├── bfly.csv
##  │   ├── bfnx.csv
##  │   ├── bnus.csv
##  │   ├── btrx.csv
##  │   ├── cbse.csv
##  │   ├── gmni.csv
##  │   ├── itbi.csv
##  │   ├── krkn.csv
##  │   ├── lmax.csv
##  │   ├── okcn.csv
##  │   └── stmp.csv
##  ├── livrable
##  │   └── livrable.ipynb
##  ├── output
##  │   ├── Export_YYYY-MM-DD_HH-MM-SS_freq-Xmin_vwmp-X
##  │   │   ├── df_all.csv
##  │   │   ├── df_bfly.csv
##  │   │   ├── df_bfnx.csv
##  │   │   ├── df_bnus.csv
##  │   │   ├── df_btrx.csv
##  │   │   ├── df_cbse.csv
##  │   │   ├── df_gmni.csv
##  │   │   ├── df_itbi.csv
##  │   │   ├── df_krkn.csv
##  │   │   ├── df_lmax.csv
##  │   │   ├── df_okcn.csv
##  │   │   ├── df_stmp.csv
##  │   │   └── synthese.csv
##  └── readme.txt